# 07.3-Step-backward-feature-selection-mlxtend
# 07.4-Step-backward-feature-selection-sklearn
### Step backward feature selection

Step Backward Feature Selection starts by fitting a model using all the features in the data set and determining its performance. 

Then, it trains models on all possible combinations of all features, minus one, and removes the feature that returns the model with the lowest performance.

In the third step, it trains models in all possible combinations of the features remaining from step 2, minus 1 feature, and removes the feature that produced the lowest performing model.

The algorithm stops when a certain criteria determined by the user is met. This criteria could be that the model performance does not decrease beyond a certain threshold, or alternatively, as we show in this notebook, when we reach a certain number of selected features.

The evaluation metric can be the roc_auc for classification or the r squared for regression, for example, and is determined by the user.

Step Backward Feature Selection is called greedy because it evaluates all possible n, and then n-1 and n-2 and so on feature combinations. Therefore, it is very computationally expensive and sometimes, if the feature space is big enough, even unfeasible.

Scikit-learn provides various stopping criteria to stop the search:

* when a certain number of features is reached (like MLXtend) (arbitrary)
* if the performance does not increase beyond a certain threshold (ideal but expensive)
* selects half of the features (arbitrary)

In [1]:
# 07.3-Step-backward-feature-selection-mlxtend


# within the SFS we indicate:
# 1) the algorithm we want to create, in this case RandomForests (note that I use few trees to speed things up)
# 2) the stopping criteria: want to select 50 features

# 3) wheter to perform step forward or step backward

# 4) the evaluation metric: in this case the roc_auc
# 5) the want cross-validation

# this is going to take a while, do not despair

sfs = SFS(RandomForestClassifier(n_estimators=10, n_jobs=4, random_state=0),
          k_features=65, # the lower the features we want, the longer this will take
          forward=False, floating=False,
          verbose=2, scoring='roc_auc',  cv=2)

sfs = sfs.fit(X_train, y_train)

NameError: name 'SFS' is not defined

# 07.4-Step-backward-feature-selection-sklearn

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector as SFS
import warnings
warnings.filterwarnings('ignore')

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Classification

In [3]:
data = pd.read_csv('../dataset_2.csv')
print(data.shape)
data.head(1)

(50000, 109)


,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109
0,4.53271,3.280834,17.982476,4.404259,2.34991,0.603264,2.784655,0.323146,12.009691,0.139346,...,2.079066,6.748819,2.941445,18.360496,17.726613,7.774031,1.473441,1.973832,0.976806,2.541417


In [4]:
X_train, X_test, y_train, y_test = train_test_split( data.drop(labels=['target'], axis=1),data['target'],test_size=0.3, random_state=0)
X_train.shape, X_test.shape

((35000, 108), (15000, 108))

### Remove correlated features

Step Backward Feature Selection takes a long time to run, so to speed it up we will reduce the feature space by removing correlated features first.

In [5]:
# remove correlated features to reduce the feature space
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr
corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  36


In [6]:
# removed correlated  features
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)
X_train.shape, X_test.shape

((35000, 72), (15000, 72))

### Step Backward Feature Selection

For the Step Backward feature selection algorithm, we are going to use the class SFS from MLXtend:
http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/

In [7]:
# within the SFS we indicate:
# 1) the algorithm we want to create, in this case RandomForests (note that I use few trees to speed things up)
# 2) the stopping criteria: check sklearn documentation for more details
# 3) wheter to perform step forward or step backward
# 4) the evaluation metric: in this case the roc_auc
# 5) the want cross-validation
# this is going to take a while, do not despair
sfs = SFS(estimator=RandomForestClassifier(n_estimators=10, n_jobs=4, random_state=0) ,
    n_features_to_select=65 ,  # the number of features to retain
    tol=None ,  # the maximum increase or decrease in the performance metric
    direction='backward' ,  # the direction of the selection procedure
    scoring='roc_auc' ,  # the metric to evaluate
    cv=2 ,  # the cross-validation fold
    n_jobs=4 ,  # for parallelization
)
sfs = sfs.fit(X_train, y_train)

In the previous log, we see that the performance does not decrease, so we could continue removing features. If you have time and patience, why don't you try that?

### Compare performance of feature subsets

In [8]:
# function to train random forests and evaluate the performance
def run_randomForests(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train set')
    pred = rf.predict_proba(X_train)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

In [9]:
selected_feat= sfs.get_feature_names_out()
selected_feat

array(['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7',
       'var_8', 'var_10', 'var_11', 'var_12', 'var_13', 'var_14',
       'var_15', 'var_16', 'var_18', 'var_19', 'var_20', 'var_21',
       'var_22', 'var_23', 'var_25', 'var_26', 'var_27', 'var_30',
       'var_31', 'var_34', 'var_35', 'var_37', 'var_40', 'var_41',
       'var_45', 'var_47', 'var_48', 'var_49', 'var_50', 'var_51',
       'var_52', 'var_53', 'var_55', 'var_56', 'var_58', 'var_60',
       'var_62', 'var_63', 'var_67', 'var_68', 'var_69', 'var_71',
       'var_73', 'var_77', 'var_78', 'var_79', 'var_81', 'var_82',
       'var_83', 'var_86', 'var_89', 'var_90', 'var_91', 'var_93',
       'var_96', 'var_98', 'var_103', 'var_107'], dtype=object)

In [10]:
# evaluate performance of algorithm built using selected features
run_randomForests(X_train[selected_feat], X_test[selected_feat], y_train, y_test)

Train set
Random Forests roc-auc: 0.7118554905228884
Test set
Random Forests roc-auc: 0.6960298383148206


In [11]:
# and for comparison, we train random forests using
# all features (except the correlated ones, which we removed already)
run_randomForests(X_train,X_test, y_train, y_test)

Train set
Random Forests roc-auc: 0.7119921185820277
Test set
Random Forests roc-auc: 0.6957598691250635


In [12]:
# Performance, as expected is roughly the same.

## Regression

Let's now repeat the process but in the context of regression. With the house prices dataset from Kaggle, the aim is to predict the continuous target: House Price.

In [13]:
data = pd.read_csv('../houseprice.csv')
print(data.shape)
data.head(1)

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [14]:
# In practice, feature selection should be done after data pre-processing, so ideally, all the categorical variables are encoded into numbers,
# and then you can assess how deterministic they are of the target
# here for simplicity I will use only numerical variables select numerical columns:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(1460, 38)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['SalePrice'], axis=1), data['SalePrice'], test_size=0.3, random_state=0)
X_train.shape, X_test.shape

((1022, 37), (438, 37))

In [16]:
# find and remove correlated features
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr
corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  3


In [17]:
# removed correlated features
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)
X_train.shape, X_test.shape

((1022, 34), (438, 34))

In [18]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

### Step Backward Feature Selection

In [19]:
# step backward feature selection algorithm
sfs = SFS( estimator = RandomForestRegressor(n_estimators=10, n_jobs=4, random_state=10), 
    n_features_to_select=20, # the number of features to retain
    tol=None, # the maximum increase or decrease in the performance metric
    direction='forward', # the direction of the selection procedure
    scoring='r2', # the metric to evaluate
    cv=2, # the cross-validation fold
    n_jobs=4, # for parallelization
)
sfs = sfs.fit(X_train, y_train)

In [20]:
sfs.get_feature_names_out()

array(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'Fireplaces', 'GarageCars', 'WoodDeckSF', '3SsnPorch',
       'ScreenPorch', 'PoolArea'], dtype=object)

### Compare performance of feature subsets

In [21]:
# function to train random forests and evaluate the performance
def run_randomForests(X_train, X_test, y_train, y_test):
    rf = RandomForestRegressor(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train set')
    pred = rf.predict(X_train)
    print('Random Forests roc-auc: {}'.format(r2_score(y_train, pred)))
    print('Test set')
    pred = rf.predict(X_test)
    print('Random Forests roc-auc: {}'.format(r2_score(y_test, pred)))

In [22]:
selected_feat= sfs.get_feature_names_out()
selected_feat

array(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'Fireplaces', 'GarageCars', 'WoodDeckSF', '3SsnPorch',
       'ScreenPorch', 'PoolArea'], dtype=object)

In [23]:
# evaluate performance of algorithm built using selected features
run_randomForests(X_train[selected_feat], X_test[selected_feat], y_train, y_test)

Train set
Random Forests roc-auc: 0.8629621682457452
Test set
Random Forests roc-auc: 0.8246015925536159


In [24]:
# and for comparison, we train random forests using all features (except the correlated ones, which we removed already)
run_randomForests(X_train,  X_test, y_train, y_test)

Train set
Random Forests roc-auc: 0.8699152317492538
Test set
Random Forests roc-auc: 0.8190809813112794


*************************
**************************
************************

# 07.5-Exhaustive-feature-selection
### Exhaustive Feature Selection
- Algorithm will evaluate all combinations and determine the best subset of features

Exhaustive Feature Selection finds the best subset of features out of all possible subsets, according to a determined performance metric for a certain machine learning algorithm.

For example, if we train a logistic regression and the dataset consists of 4 features, the algorithm will evaluate all **15** feature combinations as follows:

- all possible combinations of 1 feature

- all possible combinations of 2 features

- all possible combinations of 3 features

- all the 4 features

and select the one that results in the best performance (e.g., classification accuracy) of the logistic regression.

Exhaustive Feature Selection is a greedy algorithm as it evaluates all possible feature combinations. It is very computationally expensive and, sometimes, if the feature space is large, even unfeasible.

There is a special package for Python that implements this type of feature selection: mlxtend.

http://rasbt.github.io/mlxtend/

In the mlxtend implementation of the Exhaustive Feature Selection, the stopping criteria is an arbitrarily set number of features. So the search will finish when we reach the desired number of selected features. 

This is somewhat arbitrary, we might be selecting a sub-optimal number of features, or likewise, a high number of features. But, by looking at the performance metric returned by the algorithm as it selects the features, we can have a view on whether more features do add value, or not. 

**Note**

If we wanted to stop the search by using another criteria, we would have to code the algorithm ourselves, unfortunately :(

Here I will use the Step Forward feature selection algorithm from mlxtend in a classification and regression dataset.

In [25]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

## Classification

In [29]:
# few rows to speed things up
data = pd.read_csv('../dataset_2.csv', nrows=10000)
print(data.shape)
data.head(1)

(10000, 109)


,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109
0,4.53271,3.280834,17.982476,4.404259,2.34991,0.603264,2.784655,0.323146,12.009691,0.139346,...,2.079066,6.748819,2.941445,18.360496,17.726613,7.774031,1.473441,1.973832,0.976806,2.541417


In [30]:
X_train, X_test, y_train, y_test = train_test_split( data.drop(labels=['target'], axis=1), data['target'], test_size=0.3, random_state=0)
X_train.shape, X_test.shape

((7000, 108), (3000, 108))

### Remove correlated features

The Exhaustive Feature Selection takes a long time to run, so to speed it up we will reduce the feature space by removing correlated features first.

In [31]:
# remove correlated features to reduce the feature space
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr
# note that we reduce the correlation threshold to remove more features
corr_features = correlation(X_train, 0.6)
print('correlated features: ', len(set(corr_features)) )

correlated features:  59


In [32]:
# removed correlated features
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)
X_train.shape, X_test.shape

((7000, 49), (3000, 49))

###  Exhaustive Feature Selection

For the Exhaustive Feature Selection algorithm, we are going to use the class EFS from MLXtend:
http://rasbt.github.io/mlxtend/user_guide/feature_selection/ExhaustiveFeatureSelector/

In [33]:
# in order to shorter search time for the demonstration i will ask the algorithm to try all possible 1 and 2 feature combinations
# if you have access to a multicore or distributed computer system you can try more greedy searches
###################################
# within the EFS we indicate:
# 1) the algorithm we want to create, in this case RandomForests (note that I use few trees to speed things up)
# 2) the number of minimum features we want our model to have
# 3) the number of maximum features we want our model to have
# with 2 and 3 we regulate the number of possible feature combinations to be evaluated by the model.
# 4) the evaluation metric: in this case the roc_auc
# 5) the cross-validation
# this is going to take a while, do not despair
efs = EFS(RandomForestClassifier(n_estimators=5,  n_jobs=4,  random_state=0,  max_depth=2),
          min_features=1,
          max_features=2, scoring='roc_auc', print_progress=True, cv=2)
# search features
efs = efs.fit(X_train, y_train)

Features: 1225/1225

The log above means that the search evaluated 1225 feature combinations!

In [34]:
efs.best_idx_

(12, 32)

In [35]:
selected_feat = X_train.columns[list(efs.best_idx_)]
selected_feat

Index(['var_16', 'var_55'], dtype='object')

### Compare performance of feature subsets

In [36]:
# function to train random forests and evaluate the performance
def run_randomForests(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train set')
    pred = rf.predict_proba(X_train)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

In [37]:
# evaluate performance of classifier using selected features
run_randomForests(X_train[selected_feat], X_test[selected_feat],  y_train, y_test)

Train set
Random Forests roc-auc: 0.7152616074556793
Test set
Random Forests roc-auc: 0.6868327155531925


In [38]:
# and for comparison, we train random forests using
# all features (except the correlated ones, which we removed already)
run_randomForests(X_train, X_test, y_train, y_test)

Train set
Random Forests roc-auc: 0.7509464508443697
Test set
Random Forests roc-auc: 0.6874840935262856


Even with 2 features, the performance is not super far off of that model built using all features!!

## Regression

Let's now repeat the process but in the context of regression. With the house prices dataset from Kaggle, the aim is to predict the continuous target: House Price.

In [39]:
data = pd.read_csv('../houseprice.csv')
data.shape

(1460, 81)

In [40]:
# In practice, feature selection should be done after data pre-processing, so ideally, all the categorical variables are encoded into numbers,
# and then you can assess how deterministic they are of the target
# here for simplicity I will use only numerical variables select numerical columns:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(1460, 38)

In [41]:
X_train, X_test, y_train, y_test = train_test_split( data.drop(labels=['SalePrice'], axis=1),  data['SalePrice'], test_size=0.3, random_state=0)
X_train.shape, X_test.shape

((1022, 37), (438, 37))

In [42]:
# find and remove correlated features
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr
# note thta we reduce the threshold to remove more features
corr_features = correlation(X_train, 0.6)
print('correlated features: ', len(set(corr_features)) )

correlated features:  9


In [43]:
# removed correlated  features
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)
X_train.shape, X_test.shape

((1022, 28), (438, 28))

In [44]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

###  Exhaustive Feature Selection

In [45]:
# exhaustive search
# in order to shorter search time for the demonstration i will ask the algorithm to try all possible 10 and 11 feature combinations
# if you have access to a multicore or distributed computer system you can try more greedy searches
efs = EFS(RandomForestRegressor(n_estimators=5, n_jobs=4, random_state=0, max_depth=2),
          min_features=1,  max_features=2, scoring='r2',
          print_progress=True, cv=2)
efs = efs.fit(X_train, y_train)

Features: 406/406

In [46]:
# The searched evaluated 406 feature combinations!

In [47]:
efs.best_idx_

(4, 9)

In [48]:
X_train.columns[list(efs.best_idx_)]

Index(['OverallQual', 'BsmtFinSF1'], dtype='object')

### Compare performance of feature subsets

In [49]:
# function to train random forests and evaluate the performance
def run_randomForests(X_train, X_test, y_train, y_test):
    rf = RandomForestRegressor(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train set')
    pred = rf.predict(X_train)
    print('Random Forests roc-auc: {}'.format(r2_score(y_train, pred)))
    print('Test set')
    pred = rf.predict(X_test)
    print('Random Forests roc-auc: {}'.format(r2_score(y_test, pred)))

In [50]:
selected_feat = X_train.columns[list(efs.best_idx_)]

In [51]:
# evaluate performance of algorithm built using selected features
run_randomForests(X_train[selected_feat], X_test[selected_feat],  y_train, y_test)

Train set
Random Forests roc-auc: 0.7598962603686706
Test set
Random Forests roc-auc: 0.6856450799849569


In [52]:
# and for comparison, we train random forests using all features (except the correlated ones, which we removed already)
run_randomForests(X_train, X_test, y_train, y_test)

Train set
Random Forests roc-auc: 0.8454047044087043
Test set
Random Forests roc-auc: 0.7806708079073277


In this case, the performance decreased quite a lot, so there are additional features that are good predictors of the target.

The Exhaustive Search is extremely computationally expensive. I do not regularly use this selection procedure for this same reason. But if you have access to super computers, you may give it a go.